In [ ]:
import json
import requests
from bs4 import BeautifulSoup

url_1 = 'https://abit.itmo.ru/program/master/ai'
url_2 = 'https://abit.itmo.ru/program/master/ai_product'

urls = {
    "Искусственный интеллект": url_1,
    "Управление ИИ-продуктами": url_2
}

headers = {'User-Agent': 'Mozilla/5.0'}

for name, url in urls.items():
    try:
        # Получаем данные
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Собираем информацию о программах
        programs = {
            item.find('h5', class_='Directions_table__name__CklG5').text: ', '.join(
                f"{place.find('span').text} {place.find('p').text.lower()}"
                for place in item.find_all('div', class_='Directions_table__places__RWYBT')
            )
            for item in soup.find_all('div', class_='Directions_table__item__206L0')
        }

        # Собираем остальные данные
        data = {
            'Реализуемые программы': programs,
            'Описание программы': soup.find('div', class_='AboutProgram_aboutProgram__textWrapper___j1KQ').text.strip(),
            'Карьера': soup.find('div', class_='Career_career__container___st5X').text.strip(),
            'Вопросы и ответы': {
                q.get_text(strip=True): a.get_text(strip=True)
                for q, a in zip(
                    soup.find_all('div', class_='Accordion_accordion__title__tSP_0'),
                    soup.find_all('div', class_='Accordion_accordion__item__A6W5t')
                )
            }
        }

        # Сохраняем в JSON
        filename = f"{name}.json"
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        
        print(f"Данные успешно сохранены в файл {filename}")

    except Exception as e:
        print(f"Ошибка при обработке {name}: {str(e)}")